In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os

# Основная логика библиотеки:

In [ ]:
import os
import joblib
import numpy as np
from transformers import pipeline

# Пути к папкам с моделями
BASE_PATH = '/content/drive/My Drive/Магистерская диссертация/4 семестр/Библиотека/Модели для библиотеки'
DT_PATH = os.path.join(BASE_PATH, 'Decision Tree')
SVM_PATH = os.path.join(BASE_PATH, 'SVM')

# Тематики
THEMES = [
    'Arctic Indigenous Peoples', 'Arctic Militarization', 'Arctic Resource Extraction',
    'Arctic Shipping', 'International Governance of Arctic Regions', 'Non-Arctic Countries',
    'Permafrost', 'Science & Diplomacy', 'Svalbard'
]

# Загрузка моделей
dt_models = {}
svm_models = {}

for theme in THEMES:
    dt_models[theme] = joblib.load(os.path.join(DT_PATH, f'Decision Tree_{theme}.pkl'))
    svm_models[theme] = joblib.load(os.path.join(SVM_PATH, f'SVM_{theme}.pkl'))

# Загрузка соответствующих векторизаторов
dt_vectorizer = joblib.load(os.path.join(DT_PATH, 'Decision Tree_vectorizer.pkl'))
svm_vectorizer = joblib.load(os.path.join(SVM_PATH, 'SVM_vectorizer.pkl'))

# Загрузка zero-shot модели (без сохранения, она всегда подгружается онлайн)
zero_shot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7")


# Голосование (2 из 3)
def voting(pred_dt, pred_svm, pred_zs):
    final = []
    for i in range(len(pred_dt)):
        vote = pred_dt[i] + pred_svm[i] + pred_zs[i]
        final.append(int(vote >= 2))
    return final


# Основная функция классификации
def classify_text(text, full_output=False):
    # Векторизация текста
    text_dt_vec = dt_vectorizer.transform([text])
    text_svm_vec = svm_vectorizer.transform([text])

    # Предсказания от Decision Tree
    dt_preds = [int(model.predict(text_dt_vec)[0]) for model in dt_models.values()]

    # Предсказания от SVM
    svm_preds = [int(model.predict(text_svm_vec)[0]) for model in svm_models.values()]

    # Предсказания от zero-shot
    zs_result = zero_shot_classifier(text, candidate_labels=THEMES)
    zs_preds = [int(zs_result["labels"][0] == theme) for theme in THEMES]

    # Голосование
    final_preds = voting(dt_preds, svm_preds, zs_preds)

    # Формирование списков тем по каждому методу
    dt_labels = [THEMES[i] for i, val in enumerate(dt_preds) if val == 1] or ['Not Classified']
    svm_labels = [THEMES[i] for i, val in enumerate(svm_preds) if val == 1] or ['Not Classified']
    zs_labels = [THEMES[i] for i, val in enumerate(zs_preds) if val == 1] or ['Not Classified']
    final_labels = [THEMES[i] for i, val in enumerate(final_preds) if val == 1] or ['Not Classified']

    if full_output:
        return {
            "Decision Tree": dt_labels,
            "SVM": svm_labels,
            "Zero-Shot": zs_labels,
            "Final": final_labels
        }
    else:
        return final_labels


# Классификация текста по одной конкретной тематике (по порядковому номеру от 1 до 9).
def classify_text_single_theme(text, theme_index, full_output=False):
    theme_index -= 1
    if not (0 <= theme_index < len(THEMES)):
        raise ValueError(f"theme_index должен быть от 1 до {len(THEMES)}")

    theme = THEMES[theme_index]

    # Векторизация
    text_dt_vec = dt_vectorizer.transform([text])
    text_svm_vec = svm_vectorizer.transform([text])

    # Предсказания по одной теме
    dt_pred = int(dt_models[theme].predict(text_dt_vec)[0])
    svm_pred = int(svm_models[theme].predict(text_svm_vec)[0])

    # Zero-shot предсказание: только одна тема
    zs_result = zero_shot_classifier(text, candidate_labels=[theme])
    zs_pred = int(zs_result["labels"][0] == theme)

    # Голосование
    vote = dt_pred + svm_pred + zs_pred
    final_pred = int(vote >= 2)

    if full_output:
        return {
            "theme": theme,
            "Decision Tree": bool(dt_pred),
            "SVM": bool(svm_pred),
            "Zero-Shot": bool(zs_pred),
            "Final": bool(final_pred)
        }
    else:
        return bool(final_pred)


# Запуск, тестирование, расчет метрик:

* запуск основной библиотеки по всем текстам тестовой выборки, расчет метрик (accuracy, precision, recall, f1), построение confusion-таблиц, сохранение результатов в Excel-файлы.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import warnings
warnings.filterwarnings("ignore")

# Путь к файлу
file_path = '/content/drive/My Drive/Магистерская диссертация/4 семестр/prepared_data.xlsx'
df = pd.read_excel(file_path)

# Путь к файлу
log_file_path = '/content/drive/My Drive/Магистерская диссертация/4 семестр/Библиотека/Результаты/output.txt'

# Открываем файл и очищаем его в начале
with open(log_file_path, 'w', encoding='utf-8') as log_file:
    pass  # Просто открываем для очистки содержимого

# Функция для записи и печати
def print_and_log(message):
    print(message)  # Печать в консоль
    with open(log_file_path, 'a', encoding='utf-8') as log_file:  # Открытие файла для записи
        log_file.write(message + '\n')  # Запись в файл

# Выбор признаков и меток
X = df['full_text'].values
Y = df[THEMES]

# Делим выборку
random_state = 42
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=random_state)

# Таблицы метрик
methods = ['Decision Tree', 'SVM', 'Zero-Shot']
accuracy_table = pd.DataFrame(columns=['method'] + THEMES)
precision_table = pd.DataFrame(columns=['method'] + THEMES)
recall_table = pd.DataFrame(columns=['method'] + THEMES)
f1_table = pd.DataFrame(columns=['method'] + THEMES)

# Метрики голосования
voting_metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1': []}

# Вспомогательная функция
def print_confusion_table(y_true, y_pred, method_name, theme):
    cm = confusion_matrix(y_true, y_pred)
    if cm.shape == (2, 2):
        tn, fp, fn, tp = cm.ravel()
    else:
        tn = fp = fn = tp = 0

    confusion_df = pd.DataFrame(
        [[tp, fp], [fn, tn]],
        columns=['Релевантные (1)', 'Нерелевантные (0)'],
        index=['Найденные (1)', 'Ненайденные (0)']
    )
    print_and_log(f"\nМетод: {method_name}, Тематика: {theme}")
    print_and_log("Таблица с метриками (TP, TN, FP, FN):")
    print_and_log(confusion_df.to_string())

# Хранилища предсказаний
predictions = {method: [] for method in methods}
predictions['Final'] = []

# Хранилище истинных меток
y_true_all = Y_test.values

# Прогон по тестовой выборке
for i, text in enumerate(X_test):
    result = classify_text(text, full_output=True)

    # Истинные темы
    true_themes = [theme for j, theme in enumerate(THEMES) if Y_test.iloc[i, j] == 1]

    # Определяем формат вывода
    if isinstance(result, dict):
        dt_pred = result.get('Decision Tree', [])
        svm_pred = result.get('SVM', [])
        zs_pred = result.get('Zero-Shot', [])
        final_pred = result.get('Final', [])
    else:
        # краткий режим: result — это список тем из финального голосования
        dt_pred = []
        svm_pred = []
        zs_pred = []
        final_pred = result if result != ['Not Classified'] else []

    # Исключаем 'Not Classified' из анализа
    final_pred_clean = [theme for theme in final_pred if theme != 'Not Classified']

    # Анализ совпадений
    correct = [theme for theme in final_pred_clean if theme in true_themes]
    missed = [theme for theme in true_themes if theme not in final_pred_clean]
    extra = [theme for theme in final_pred_clean if theme not in true_themes]

    # Вывод и запись в файл
    print_and_log(f"\n--- Текст #{i} ---")
    print_and_log(f"Правильные темы: {true_themes}")
    print_and_log(f"Предсказание:")
    if isinstance(result, dict):
        print_and_log(f"Decision Tree: {dt_pred} | SVM: {svm_pred} | Zero-Shot: {zs_pred}")
    print_and_log(f"Final: {final_pred_clean if final_pred_clean else ['Not Classified']}")

    print_and_log(f"Угаданы: {correct} | Пропущены: {missed} | Лишние: {extra}")

    # Сбор предсказаний
    for method in methods:
        pred = [1 if theme in result.get(method, []) else 0 for theme in THEMES] if isinstance(result, dict) else [0] * len(THEMES)
        predictions[method].append(pred)

    final_pred_vector = [1 if theme in final_pred_clean else 0 for theme in THEMES]
    predictions['Final'].append(final_pred_vector)


# Преобразуем в массивы
for key in predictions:
    predictions[key] = np.array(predictions[key])

# Расчет метрик по методам
for method in methods:
    row_acc = [method]
    row_prec = [method]
    row_rec = [method]
    row_f1 = [method]

    print_and_log(f"\nМетод: {method}")
    for idx, theme in enumerate(THEMES):
        y_true = y_true_all[:, idx]
        y_pred = predictions[method][:, idx]

        accuracy = round(accuracy_score(y_true, y_pred), 2)
        precision = round(precision_score(y_true, y_pred, zero_division=0), 2)
        recall = round(recall_score(y_true, y_pred, zero_division=0), 2)
        f1 = round(f1_score(y_true, y_pred, zero_division=0), 2)

        row_acc.append(accuracy)
        row_prec.append(precision)
        row_rec.append(recall)
        row_f1.append(f1)

        print_confusion_table(y_true, y_pred, method, theme)

    accuracy_table.loc[len(accuracy_table)] = row_acc
    precision_table.loc[len(precision_table)] = row_prec
    recall_table.loc[len(recall_table)] = row_rec
    f1_table.loc[len(f1_table)] = row_f1

# Метрики голосования
for idx, theme in enumerate(THEMES):
    y_true = y_true_all[:, idx]
    y_pred = predictions['Final'][:, idx]

    accuracy = round(accuracy_score(y_true, y_pred), 2)
    precision = round(precision_score(y_true, y_pred, zero_division=0), 2)
    recall = round(recall_score(y_true, y_pred, zero_division=0), 2)
    f1 = round(f1_score(y_true, y_pred, zero_division=0), 2)

    voting_metrics['accuracy'].append(accuracy)
    voting_metrics['precision'].append(precision)
    voting_metrics['recall'].append(recall)
    voting_metrics['f1'].append(f1)

    print_confusion_table(y_true, y_pred, "Voting", theme)

# Сохраняем метрики в Excel
results_path = '/content/drive/My Drive/Магистерская диссертация/4 семестр/Библиотека/Результаты'
accuracy_table.to_excel(f'{results_path}/accuracy_methods.xlsx', index=False)
precision_table.to_excel(f'{results_path}/precision_methods.xlsx', index=False)
recall_table.to_excel(f'{results_path}/recall_methods.xlsx', index=False)
f1_table.to_excel(f'{results_path}/f1_methods.xlsx', index=False)

voting_df = pd.DataFrame(voting_metrics, index=THEMES).T.reset_index()
voting_df.rename(columns={'index': 'metric'}, inplace=True)
voting_df.to_excel(f'{results_path}/voting_summary.xlsx', index=False)


In [ ]:
import pandas as pd

# Путь к файлу с текстами
file_path = '/content/drive/My Drive/Магистерская диссертация/4 семестр/prepared_data.xlsx'
df = pd.read_excel(file_path)
text = df.iloc[28, 0]

detailed = classify_text(text, full_output=True)

print("--- Текст #1 ---")
print("Предсказание:")
print(f"Decision Tree: {detailed['Decision Tree']} | SVM: {detailed['SVM']} | Zero-Shot: {detailed['Zero-Shot']}")
print("Final:", detailed['Final'])

## Тестирование классификации текста по одной конкретной тематике:

In [ ]:
import pandas as pd

# Путь к файлу с текстами
file_path = '/content/drive/My Drive/Магистерская диссертация/4 семестр/prepared_data.xlsx'
df = pd.read_excel(file_path)
text = df.iloc[28, 0]

theme_index = 7  # Permafrost

# Краткий результат (True/False)
result = classify_text_single_theme(text, theme_index)
print("Итоговая классификация:", result)

# Подробный результат
detailed_result = classify_text_single_theme(text, theme_index, full_output=True)
print("\nПодробно:")
for k, v in detailed_result.items():
    print(f"{k}: {v}")
